Задачи:

- ~~Извлекать статьи по ключевым словам;~~
- ~~Суммаризировать каждую статью;~~
- Суммаризация подходов из статей;
- Рерайтинг запроса;
- Статья в виде json;
- Выбор подходящей LLM;
- Разработка веб-интерфейса;
- *Попробовать RAPTOR для суммаризации (https://arxiv.org/html/2401.18059v1);

In [3]:
# from langchain import OpenAI, LLMAgent
from langchain.agents import initialize_agent, tool
from langchain.agents import AgentExecutor, create_gigachat_functions_agent
from langchain.chat_models.gigachat import GigaChat
import arxiv
import os

In [4]:
llm = GigaChat(credentials=os.environ.get("SB_AUTH_DATA"),
                model="GigaChat", timeout=30, verify_ssl_certs=False)

In [5]:
# Инструмент поиска на Arxiv
@tool("ArxivSearch") # name="arxiv_search", description=""
def arxiv_search_tool(query: str, max_results: int = 2):
    """Search articles on Arxiv by keywords"""
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    articles = []
    for result in search.results():
        articles.append({
            "title": result.title,
            "summary": result.summary,
            "url": result.entry_id
        })
    return articles

# Инструмент суммаризации текста
@tool("SummarizingTool") # name="summarize_tool", description="Summarizes text using a language model"
def summarize_tool(text: str):
    """Summarizes text using a language model"""
    prompt = f"Summarize the following article: {text}"
    summary = llm(prompt)
    return summary



In [6]:
# Инициализация агента с инструментами
tools = [arxiv_search_tool, summarize_tool]
# agent = initialize_agent(
#     tools=tools,
#     llm=llm,
#     # agent=LLMAgent(name="ArxivSummarizerAgent")
# )
agent = create_gigachat_functions_agent(llm, tools)
executor = AgentExecutor(agent=agent, tools=tools)

In [11]:
# Пример использования агента
def search_and_summarize_arxiv(query):
    articles = executor({"input": query})
    return articles

# Запуск
query = "Summarize articles about attention"
result = search_and_summarize_arxiv(query)
print(result)

C:\Users\koltu\AppData\Local\Temp\ipykernel_29732\724633227.py:11: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


{'input': 'Summarize articles about attention', 'output': 'Articles about attention often highlight the importance of attention mechanisms in visual question answering (VQA) tasks. They discuss how these mechanisms help models focus on relevant areas of images and textual information, leading to improved performance. One article proposes a method to generate human-like attention maps using a Human Attention Network (HAN) trained on the VQA-HAT dataset. Another article introduces Agent Attention, a novel attention paradigm that balances computational efficiency and representation power by introducing agent tokens to the attention module. This new approach is shown to integrate the benefits of Softmax attention and linear attention, offering significant efficiency gains without sacrificing expressiveness. Both articles present promising directions for improving attention mechanisms in VQA and other vision tasks.'}


In [12]:
print(result['output'])

Articles about attention often highlight the importance of attention mechanisms in visual question answering (VQA) tasks. They discuss how these mechanisms help models focus on relevant areas of images and textual information, leading to improved performance. One article proposes a method to generate human-like attention maps using a Human Attention Network (HAN) trained on the VQA-HAT dataset. Another article introduces Agent Attention, a novel attention paradigm that balances computational efficiency and representation power by introducing agent tokens to the attention module. This new approach is shown to integrate the benefits of Softmax attention and linear attention, offering significant efficiency gains without sacrificing expressiveness. Both articles present promising directions for improving attention mechanisms in VQA and other vision tasks.
